In [47]:
import os

from dotenv import load_dotenv, find_dotenv

from llm_agent.utils.hydra import get_agent_config
from llm_agent.utils.paths import PROMPTS_CONFIG_PATH
_ = load_dotenv(find_dotenv("../secrets.env")) # read local .env file

import warnings
warnings.filterwarnings("ignore")


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')

In [48]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [49]:
# load the document and split it into chunks
loader = TextLoader("documents/articulo_67_consittucion.txt")
document1 = loader.load()

loader = TextLoader("documents/articulo_155_constitucion.txt")
document2 = loader.load()

docs = document1 + document2

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# 1. Qdrant

In [50]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [51]:
client = QdrantClient(
    url="https://e55749a7-835b-40d8-af26-fde2f9cbfbb4.europe-west3-0.gcp.cloud.qdrant.io",
    api_key=QDRANT_API_KEY
)

In [52]:
client.recreate_collection(
    collection_name="boe",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

True

In [40]:
# Local db

db = Qdrant.from_documents(
    docs, embedding=OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY')), 
    location=":memory:",  # Local mode with in-memory storage only
)

In [53]:
# Cloud db

db = Qdrant(
    client=QdrantClient(
        url="https://e55749a7-835b-40d8-af26-fde2f9cbfbb4.europe-west3-0.gcp.cloud.qdrant.io",
        api_key=QDRANT_API_KEY
    ),
    collection_name="boe",
    embeddings=OpenAIEmbeddings(
        api_key=OPENAI_API_KEY
    )
)

db.add_documents(docs)

['bea5f939e4d54338b02fb92bf0264130', '10381b41c8754a26becf57d82a633ffa']

In [54]:
query = "Cual es al articulo 155?"
docs = db.similarity_search(query)

In [55]:
docs[0].page_content

'ArtÃ\xadculo 155\n\nSi una Comunidad AutÃ³noma no cumpliere las obligaciones que la ConstituciÃ³n u otras leyes le impongan, o actuare de forma que atente gravemente al interÃ©s general de EspaÃ±a, el Gobierno, previo requerimiento al Presidente de la Comunidad AutÃ³noma y, en el caso de no ser atendido, con la aprobaciÃ³n por mayorÃ\xada absoluta del Senado, podrÃ¡ adoptar las medidas necesarias para obligar a aquÃ©lla al cumplimiento forzoso de dichas obligaciones o para la protecciÃ³n del mencionado interÃ©s general.\nPara la ejecuciÃ³n de las medidas previstas en el apartado anterior, el Gobierno podrÃ¡ dar instrucciones a todas las autoridades de las Comunidades AutÃ³nomas.\n'

# 2. Chroma

In [56]:
from langchain_community.vectorstores import Chroma

In [57]:
db = Chroma.from_documents(
    docs, embedding=OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
)

In [58]:
query = "Cual es el articulo 67?"
docs = db.similarity_search(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [59]:
docs[0].page_content

'ArtÃ\xadculo 67\n\n1. Nadie podrÃ¡ ser miembro de las dos CÃ¡maras simultÃ¡neamente, ni acumular el acta de una Asamblea de Comunidad AutÃ³noma con la de Diputado al Congreso.\n2. Los miembros de las Cortes Generales no estarÃ¡n ligados por mandato imperativo.\n3. Las reuniones de Parlamentarios que se celebren sin convocatoria reglamentaria no vincularÃ¡n a las CÃ¡maras, y no podrÃ¡n ejercer sus funciones ni ostentar sus privilegios.\n'